In [1]:
from tempo.serve.metadata import ModelFramework
from tempo.serve.model import Model
from tempo.seldon.docker import SeldonDockerRuntime
from tempo.kfserving.protocol import KFServingV2Protocol
from tempo.serve.utils import pipeline, predictmethod
from tempo.seldon.k8s import SeldonKubernetesRuntime

import numpy as np
import os 
import pprint

In [2]:
sklearn_model = Model(
        name="test-iris-sklearn",
        runtime=SeldonDockerRuntime(),
        platform=ModelFramework.SKLearn,
        uri="gs://seldon-models/sklearn/iris",
        local_folder=os.getcwd()+"/sklearn")

xgboost_model = Model(
        name="test-iris-xgboost",
        runtime=SeldonDockerRuntime(),
        platform=ModelFramework.XGBoost,
        uri="gs://seldon-models/xgboost/iris",
        local_folder=os.getcwd()+"/xgboost")

@pipeline(name="mypipeline",
          runtime=SeldonDockerRuntime(protocol=KFServingV2Protocol("test")),
          uri="gs://seldon-models/custom",
          models=[sklearn_model, xgboost_model])
class MyPipeline(object):

    def __init__(self):
        self.counter = 0

    @predictmethod
    def predict(self, payload: np.ndarray) -> np.ndarray:
        self.counter += 1
        res1 = sklearn_model(payload)
        
        if res1[0][0] > 0.7:
            return res1
        else:
            return xgboost_model(payload)
        

    def get_counter(self):
        return self.counter

In [3]:
sklearn_model.download()
xgboost_model.download()

2021/02/13 10:02:18 ERROR : : Entry doesn't belong in directory "" (same as directory) - ignoring

2021/02/13 10:02:19 ERROR : : Entry doesn't belong in directory "" (same as directory) - ignoring



In [4]:
myp = MyPipeline()

In [6]:
myp.deploy()

In [7]:
myp.predict(np.array([[4.9, 3.1, 1.5, 0.2]]))

array([[8.09104104e-01, 1.90862024e-01, 3.38721089e-05]])

In [8]:
myp.undeploy()

In [9]:
sklearn_model._runtime = SeldonKubernetesRuntime()
yaml = sklearn_model.to_k8s_yaml()
print (eval(pprint.pformat(yaml)))

/home/clive/anaconda3/envs/mlops/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: test-iris-sklearn
  namespace: default
spec:
  predictors:
  - graph:
      implementation: SKLEARN_SERVER
      modelUri: gs://seldon-models/sklearn/iris
      name: classifier
    name: default
    replicas: 1

